In [28]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from joblib import dump

In [ ]:
"""
Primary:
- id (Id of track generated by Spotify)
Numerical:
- acousticness (Ranges from 0 to 1)
- danceability (Ranges from 0 to 1)
- energy (Ranges from 0 to 1)
- duration_ms (Integer typically ranging from 200k to 300k)
- instrumentalness (Ranges from 0 to 1)
- valence (Ranges from 0 to 1)
- popularity (Ranges from 0 to 100)
- tempo (Float typically ranging from 50 to 150)
- liveness (Ranges from 0 to 1)
- loudness (Float typically ranging from -60 to 0)
- speechiness (Ranges from 0 to 1)
- year (Ranges from 1921 to 2020)
Dummy:
- mode (0 = Minor, 1 = Major)
- explicit (0 = No explicit content, 1 = Explicit content)
Categorical:
- key (All keys on octave encoded as values ranging from 0 to 11, starting on C as 0, C# as 1 and so on…)
- artists (List of artists mentioned)
- release_date (Date of release mostly in yyyy-mm-dd format, however precision of date may vary)
- name (Name of the song)"""

In [16]:
# reading csv
df = pd.read_csv('data/data.csv')
# deciding on columns(subset removed 'id' 'release date') reordered columns
df = df[['artists',  'name', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'popularity', 'speechiness', 'tempo', 'valence']]

In [23]:
# tiny amount of data cleaning
df['artists'] = df['artists'].str.strip('[]')
df['artists'] = df['artists'].str.strip('""')
df['artists'] = df['artists'].str.strip("''")

In [24]:
df.head(20)

,artists,name,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,Mamie Smith,Keep A Song In Your Soul,1920,0.991000,0.598,168333,0.2240,0,0.000522,5,0.3790,-12.628,0,12,0.0936,149.976,0.6340
1,Screamin' Jay Hawkins,I Put A Spell On You,1920,0.643000,0.852,150200,0.5170,0,0.026400,5,0.0809,-7.261,0,7,0.0534,86.889,0.9500
2,Mamie Smith,Golfing Papa,1920,0.993000,0.647,163827,0.1860,0,0.000018,0,0.5190,-12.098,1,4,0.1740,97.600,0.6890
3,Oscar Velazquez,True House Music - Xavier Santos & Carlos Gomi...,1920,0.000173,0.730,422087,0.7980,0,0.801000,2,0.1280,-7.311,1,17,0.0425,127.997,0.0422
4,Mixe,Xuniverxe,1920,0.295000,0.704,165224,0.7070,1,0.000246,10,0.4020,-6.036,0,2,0.0768,122.076,0.2990
5,Mamie Smith & Her Jazz Hounds,Crazy Blues - 78rpm Version,1920,0.996000,0.424,198627,0.2450,0,0.799000,5,0.2350,-11.470,1,9,0.0397,103.870,0.4770
6,Mamie Smith,Don't You Advertise Your Man,1920,0.992000,0.782,195200,0.0573,0,0.000002,5,0.1760,-12.453,1,5,0.0592,85.652,0.4870
7,Mamie Smith & Her Jazz Hounds,Arkansas Blues,1920,0.996000,0.474,186173,0.2390,0,0.186000,9,0.1950,-9.712,1,0,0.0289,78.784,0.3660
8,Francisco Canaro,La Chacarera - Remasterizado,1920,0.996000,0.469,146840,0.2380,0,0.960000,8,0.1490,-18.717,1,0,0.0741,130.060,0.6210
9,Meetya,Broken Puppet - Original Mix,1920,0.006820,0.571,476304,0.7530,0,0.873000,8,0.0920,-6.943,1,0,0.0446,126.993,0.1190


In [262]:
# finding a song I want to use as an input
frank = df[df['artists'] == 'Frank Ocean']

In [264]:
frank.head(20)

,artists,name,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
18130,Frank Ocean,Novacane,2011,0.05840,0.762,302347,0.5080,1,0.001260,3,0.1600,-9.112,1,70,0.0919,93.510,0.3700
18278,Frank Ocean,Thinkin Bout You,2012,0.40900,0.721,200747,0.3390,0,0.001530,0,0.0973,-11.195,1,73,0.0532,129.830,0.2000
18296,Frank Ocean,Lost,2012,0.02720,0.913,234093,0.6030,1,0.000503,8,0.1670,-4.892,1,71,0.2260,123.061,0.4970
19082,Frank Ocean,Nights,2016,0.42000,0.466,307151,0.5480,1,0.000001,5,0.1130,-9.362,0,78,0.1180,89.815,0.4230
19084,Frank Ocean,Ivy,2016,0.78200,0.567,249191,0.3880,1,0.000309,9,0.2480,-9.579,0,79,0.0384,116.362,0.4520
19090,Frank Ocean,Pink + White,2016,0.67000,0.544,184516,0.5520,0,0.000046,9,0.4150,-7.450,1,79,0.0991,159.738,0.5540
19092,Frank Ocean,Godspeed,2016,0.93100,0.421,177922,0.0952,0,0.000201,6,0.1260,-12.561,1,79,0.0479,109.698,0.0773
19094,Frank Ocean,White Ferrari,2016,0.78400,0.444,248808,0.0924,0,0.000000,0,0.3040,-15.605,1,77,0.0366,108.917,0.2190
19148,Frank Ocean,Self Control,2016,0.76500,0.572,249668,0.2090,0,0.000000,8,0.3560,-10.413,1,74,0.0313,80.069,0.4460
19272,Frank Ocean,Chanel,2017,0.87400,0.776,210285,0.5030,1,0.000000,0,0.1120,-5.732,0,81,0.2370,110.134,0.4730


In [149]:
# target set will be both artist and name
y_set = ['artists', 'name']

# droping target from data matrix
df_data = df.drop(y_set, axis=1)

# set target
df_target = df[y_set]

In [273]:
# fit on data, 12 neighbors
nn = NearestNeighbors(algorithm='brute', leaf_size =15, n_neighbors=12, n_jobs=-1)
nn.fit(df_data)

NearestNeighbors(algorithm='brute', leaf_size=15, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=12, p=2,
                 radius=1.0)

In [295]:
# sample a song(index) from df_data to use as our query point 
input_index = 19272 # Frank Ocean - Chanel

# vectorize 
data_vect = [df_data.iloc[input_index].values]
data_vect

[array([ 2.01700e+03,  8.74000e-01,  7.76000e-01,  2.10285e+05,
         5.03000e-01,  1.00000e+00,  0.00000e+00,  0.00000e+00,
         1.12000e-01, -5.73200e+00,  0.00000e+00,  8.10000e+01,
         2.37000e-01,  1.10134e+02,  4.73000e-01])]

In [296]:
# Query Using kneighbors 
neigh_dist, neigh_indices = nn.kneighbors(data_vect)

In [297]:
# top 12 closest data vectors to our reference vector, data_vect
neigh_dist

array([[2.76213586e-03, 2.34324959e+01, 2.63210405e+01, 3.07716034e+01,
        3.14111451e+01, 3.22390653e+01, 3.26108410e+01, 3.66270893e+01,
        3.74952521e+01, 3.89251496e+01, 3.96217930e+01, 4.31334181e+01]])

In [298]:
# these are the corresponding indicies of the most similar vectors 
neigh_indices

array([[ 19272, 125795,  92458, 107269,  34993,  56606,  17838, 142383,
         13710,  38602, 171270, 122833]], dtype=int64)

In [299]:
# breaking down the array to a list
indexs = neigh_indices.flat[0:12].tolist()
indexs

[19272,
 125795,
 92458,
 107269,
 34993,
 56606,
 17838,
 142383,
 13710,
 38602,
 171270,
 122833]

In [300]:
# result metrics
df_data.iloc[indexs]

,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
19272,2017,0.87400,0.776,210285,0.503,1,0.000000,0,0.1120,-5.732,0,81,0.2370,110.134,0.473
125795,2016,0.59100,0.488,210293,0.762,0,0.000000,8,0.0763,-3.842,1,66,0.0865,123.889,0.709
92458,2013,0.07300,0.722,210293,0.821,0,0.000000,1,0.6310,-3.856,0,64,0.1650,127.944,0.721
107269,2004,0.07300,0.760,210293,0.770,0,0.000000,5,0.1090,-6.797,0,55,0.0990,113.272,0.895
34993,1998,0.04310,0.677,210267,0.897,0,0.000000,2,0.2640,-5.458,1,64,0.0393,107.780,0.828
56606,2011,0.00379,0.696,210267,0.546,0,0.000004,5,0.3320,-6.550,1,65,0.0414,130.002,0.787
17838,2009,0.02420,0.855,210307,0.668,0,0.000000,11,0.1020,-4.892,1,69,0.0644,125.846,0.803
142383,2017,0.85300,0.389,210308,0.308,0,0.937000,7,0.1720,-14.460,0,63,0.0751,129.138,0.130
13710,1989,0.59600,0.649,210267,0.618,0,0.000001,9,0.1920,-10.728,1,68,0.0265,114.688,0.667
38602,2016,0.74300,0.707,210320,0.656,0,0.000000,4,0.1720,-9.279,1,67,0.1730,118.037,0.748


In [301]:
# result target
df_target.iloc[indexs]

,artists,name
19272,Frank Ocean,Chanel
125795,John Legend,Love Me Now
92458,Selena Gomez,Slow Down
107269,The Jacksons,Blame It on the Boogie
34993,Shakira,Si Te Vas
56606,Britney Spears,I Wanna Go
17838,"Jay Sean', 'Sean Paul', 'Lil Jon",Do You Remember
142383,Hippie Sabotage,Drifter
13710,Roy Orbison,You Got It
38602,"Christopher Jackson', 'Rachel House', 'Nicole ...",Where You Are
